In [22]:
import inro.modeller as m
import os
import glob
import csv

mm = m.Modeller()
eb = m.Modeller().emmebank

#tools
import_matrices = mm.tool("tmg.input_output.import_binary_matrix")           
change_matrix = mm.tool("inro.emme.data.matrix.change_matrix_properties")
create_matrix =  mm.tool("inro.emme.data.matrix.create_matrix")
matrix_calculator = mm.tool("inro.emme.matrix_calculation.matrix_calculator")
matrix_transaction = mm.tool('inro.emme.data.matrix.matrix_transaction')

#HDR Scripts
common = mm.module('org.emme.common')
moe = mm.module('org.emme.moe')

#scenario folder names
sc_2011 = "2011 GGHM validation run"
sc_2041_base = "2041\\GGHM Base Case"
sc_2016 = "2016"
sc_2041_C_plus_w_GTAWest = "2041\\MTP_304"

## zone grouping definitions ## 

sheds = [ "gf1","gs10;gs11","gs20;gs21","gs30;gs31", "gs40;gs41", "gs50;gs51","gs60;gs61", "gs70;gs71"]
    
#assumes all auto centroid connectors have vdf = 90 exclusively

#auto occupancy
sov_oc = 1.2
hov2_oc = 2.2
hov3_oc = 3.3

#number of hours for transit assignment
num_hrs_am = 2
#num_hrs_md = 6

In [14]:
#define inputs

scenario_folder = sc_2016
sc_num = 109

#input folder
matrix_folder = u"P:\\451010\\281829_GGH\\GGHM Runs\\" + scenario_folder

# demand
# peak period - 7 hrs
sov_demand_pk = "auto_sov_PkPd"
hov2_demand_pk = "auto_hov2_PkPd"
hov3_demand_pk = "auto_hov3_PkPd"
bike_demand_pk = "Bike_PkPd"
walk_demand_pk = "Walk_PkPd"
transit_demand_pk = "PnA_Transit_PkPd"


#offpeak
sov_demand_offpk = "auto_sov_offPkPd"
hov2_demand_offpk = "auto_hov2_offPkPd"
hov3_demand_offpk = "auto_hov3_offPkPd"
bike_demand_offpk = "Bike_offPkPd"
walk_demand_offpk = "Walk_offPkPd"
transit_demand_offpk = "PnA_Transit_OffPeakPd"

In [15]:
#delete all FULL matrices
common.clear_matrices(eb)

matrix_list = {
    sov_demand_pk : "",
    hov2_demand_pk : "",
    hov3_demand_pk : "",
    bike_demand_pk : "",
    walk_demand_pk : "",
    transit_demand_pk : "",
    sov_demand_offpk : "",
    hov2_demand_offpk : "",
    hov3_demand_offpk : "",
    bike_demand_offpk : "",
    walk_demand_offpk : "",
    transit_demand_offpk : ""
    }

#Import matrices
for matrix_name in matrix_list.keys():
    matrix_list[matrix_name] = eb.available_matrix_identifier('FULL')
    create_matrix(matrix_id = matrix_list[matrix_name],
                  matrix_name= matrix_name,
                  matrix_description = matrix_name)

common.import_matrices_from_directory(matrix_folder, matrix_list, ".mdf",sc_num) 
    
# Make total trip matrix

total_trip_matrix = eb.available_matrix_identifier('FULL')
create_matrix(matrix_id = total_trip_matrix,
            matrix_name= "daily.trips.all_modes",
            matrix_description = "daily.trips.all_modes")

exp_total_trips = matrix_list[sov_demand_pk] + "+" + matrix_list[sov_demand_pk] + "'+"  \
    + matrix_list[hov2_demand_pk] + "+" + matrix_list[hov2_demand_pk] + "'+"  \
    + matrix_list[hov3_demand_pk] + "+" + matrix_list[hov3_demand_pk] + "'+"  \
    + matrix_list[bike_demand_pk] + "+" + matrix_list[bike_demand_pk] + "'+"  \
    + matrix_list[walk_demand_pk] + "+" + matrix_list[walk_demand_pk] + "'+"  \
    + matrix_list[transit_demand_pk] + "+" + matrix_list[transit_demand_pk] + "'+"  \
    + matrix_list[sov_demand_offpk] + "+" \
    + matrix_list[hov2_demand_offpk] + "+" \
    + matrix_list[hov3_demand_offpk] + "+" \
    + matrix_list[bike_demand_offpk] + "+" \
    + matrix_list[walk_demand_offpk] + "+" \
    + matrix_list[transit_demand_offpk]
    
spec = common.get_mc_spec(total_trip_matrix, exp_total_trips ,None,None,"all", "all" ,None)
matrix_calculator(spec)

Imported matrix mf6
Imported matrix mf12
Imported matrix mf11
Imported matrix mf4
Imported matrix mf8
Imported matrix mf10
Imported matrix mf5
Imported matrix mf7
Imported matrix mf9
Imported matrix mf2
Imported matrix mf3
Imported matrix mf1
Finished Importing Matrices


In [26]:
out_file = "C:\\Users\\mnasterska\\Documents\\Projects\\GGH\\Shed_data.csv"

with open(out_file, "w") as shedf:
    shedf.write(scenario_folder +",")
    for column in sheds:
        shedf.write(column +",")
    shedf.write("\n")
    for origin in sheds:
        shedf.write(origin +",")
        for destination in sheds:
            spec = common.get_mc_spec(None, total_trip_matrix ,"+","+",origin, destination ,None)
            shedf.write( str(matrix_calculator(spec)['result']) + ',')
        shedf.write("\n")